In [ ]:
import time
import os
import pandas as pd
import datetime
import sqlite3
import glob


def table_exist(conn, table):
    return list(conn.execute(
        "select count(*) from sqlite_master where type='table' and name='" + table + "'"))[0][0] == 1


def add_to_sql(conn, name, df):

    # get the existing dataframe in sqlite3
    exist = table_exist(conn, name)
    ret = pd.read_sql('select * from ' + name, conn,
                      index_col=['stock_id', 'date']) if exist else pd.DataFrame()

    # add new df to the dataframe
    ret = ret.append(df)
    ret.reset_index(inplace=True)
    ret['stock_id'] = ret['stock_id'].astype(str)
    ret['date'] = pd.to_datetime(ret['date'])
    ret = ret.drop_duplicates(['stock_id', 'date'], keep='last')
    ret = ret.sort_values(['stock_id', 'date']).set_index(['stock_id', 'date'])

    # add the combined table
    ret.to_csv('backup.csv')

    try:
        ret.to_sql(name, conn, if_exists='replace')
    except:
        ret = pd.read_csv('backup.csv', parse_dates=[
                          'date'], dtype={'stock_id': str})
        ret['stock_id'] = ret['stock_id'].astype(str)
        ret.set_index(['stock_id', 'date'], inplace=True)
        ret.to_sql(name, conn, if_exists='replace')


def clean_to_sql(df):
    ret = df
    ret.reset_index(inplace=True)
    ret['stock_id'] = ret['stock_id'].astype(str)
    ret['date'] = pd.to_datetime(ret['date'])
    ret = ret.drop_duplicates(['stock_id', 'date'], keep='last')
    ret = ret.sort_values(['stock_id', 'date']).set_index(['stock_id', 'date'])
    return ret


def csv_to_sql(path, conn, table_name):
    csv_files = glob.glob(path)
    for file in csv_files:
        df = pd.read_csv(file)
        df = df.set_index(['stock_id', 'date'])
        add_to_sql(conn, table_name, df)
        print('target: ' + os.path.basename(file))
        print('save')



NameError: ignored

In [ ]:
root_path = 'drive/My Drive/股票/'

In [ ]:
conn = sqlite3.connect(os.path.join(root_path + 'data', 'data.db'))
csv_to_sql(root_path + '三大法人買賣超/*.csv', conn, 'legal_person')

In [ ]:
csv_files = glob.glob(root_path + '三大法人買賣超/*.csv')
dfs = pd.DataFrame()
for file in csv_files:
    dfs = dfs.append(pd.read_csv(file))
# dfs = clean_to_sql(dfs)
# dfs = dfs.set_index(['stock_id', 'date'])



index   外資買進股數  ...  自營商買賣超股數(避險)  stock_id
stock_id date                                        ...                        
0015     2012-05-04 15:51:47.610236    382   8000.0  ...           NaN         1
         2012-05-08 15:51:47.610236    249   1000.0  ...           NaN         1
         2012-05-09 12:15:38.768543     91  77000.0  ...           NaN         1
         2012-05-10 12:15:38.768543    240   6000.0  ...           NaN         1
         2012-05-11 12:15:38.768543    126   7000.0  ...           NaN         1
...                                    ...      ...  ...           ...       ...
9958     2020-06-18 12:15:38.768543    912      NaN  ...      -53000.0         1
         2020-06-19 12:15:38.768543    911      NaN  ...      -82000.0         1
         2020-06-22 12:15:38.768543    169      NaN  ...       47000.0         1
         2020-06-23 12:15:38.768543    983      NaN  ...     -137000.0         1
         2020-06-24 12:15:38.768543    844      NaN  ...      -52000.0         1

[1708320 rows x 24 columns]